In [1]:
from Bio import SeqIO

def parse_dictionary(dict_file):
    """
    Parse the dictionary file to create a mapping of VSG IDs to their classes.
    Returns a dictionary: {vsg_id: class_info}
    """
    vsg_dict = {}
    
    with open(dict_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                # Remove the '>' character
                line = line[1:]
                # Split by space to separate ID and class
                parts = line.split()
                if len(parts) >= 2:
                    vsg_id = parts[0]  # e.g., "Tb427VSG-2"
                    class_info = parts[1]  # e.g., "BES" or "array_Chr9_5A_Tb427v10"
                    vsg_dict[vsg_id] = class_info
    
    return vsg_dict

def add_class_to_fasta(input_fasta, dict_file, output_fasta):
    """
    Read FASTA file and add class information to sequence IDs.
    Creates a new FASTA file with format: >{original_id}_{class}
    """
    # Parse the dictionary file
    vsg_dict = parse_dictionary(dict_file)
    
    # Process the FASTA file
    modified_records = []
    not_found = []
    
    for record in SeqIO.parse(input_fasta, "fasta"):
        original_id = record.id
        
        # Check if this ID exists in the dictionary
        if original_id in vsg_dict:
            # Add class information to the ID
            class_info = vsg_dict[original_id]
            record.id = f"{original_id}_{class_info}"
            record.description = f"{original_id}_{class_info}"
            modified_records.append(record)
        else:
            # Keep original if not found in dictionary
            not_found.append(original_id)
            modified_records.append(record)
    
    # Write the modified records to a new FASTA file
    SeqIO.write(modified_records, output_fasta, "fasta")
    
    print(f"Processed {len(modified_records)} sequences")
    print(f"Modified {len(modified_records) - len(not_found)} sequences with class information")
    
    if not_found:
        print(f"\nWarning: {len(not_found)} sequences not found in dictionary:")
        for id in not_found[:10]:  # Show first 10
            print(f"  - {id}")
        if len(not_found) > 10:
            print(f"  ... and {len(not_found) - 10} more")

# Example usage
if __name__ == "__main__":
    input_fasta = "VSGs_Sequences.fa"  # Replace with your FASTA file name
    dict_file = "vsg_dic.txt"
    output_fasta = "VSGs_Sequences_with_class.fasta"
    
    add_class_to_fasta(input_fasta, dict_file, output_fasta)

Processed 298 sequences
Modified 298 sequences with class information


In [3]:
count_dict = {}
for n in open('VSGs_Sequences_with_class.fasta'):
    if n.startswith('>'):
        vsg_class = n.split('_')[1].strip()
        if vsg_class in count_dict:
            count_dict[vsg_class]+=1
        else:
            count_dict[vsg_class]=1
count_dict

{'BES': 14, 'MES': 5, 'MC': 62, 'array': 217}

In [6]:
#!mmseqs easy-cluster VSGs_Sequences.fa vsgsClusters/res_09_1 tmp --min-seq-id 0.9 -c 1 --cov-mode 1

Create directory tmp
easy-cluster VSGs_Sequences.fa vsgsClusters/res_09_1 tmp --min-seq-id 0.9 -c 1 --cov-mode 1 

MMseqs Version:                     	18-8cc5c
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	4
k-mer length                        	0
Target search mode                  	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	1
Coverage mode                       	1
Compositional bias                  	1
Compositional bias scale            	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mask residues          

In [7]:
!mkdir -p vsgsSearch
!mmseqs easy-search --search-type 3 --format-mode 4 VSGs_Sequences_with_class.fasta TriTrypDB-68_TbruceiLister427_2018_Genome.fasta vsgsSearch/res_vsg.m8 tmp

Create directory tmp
easy-search --search-type 3 --format-mode 4 VSGs_Sequences_with_class.fasta TriTrypDB-68_TbruceiLister427_2018_Genome.fasta vsgsSearch/res_vsg.m8 tmp 

MMseqs Version:                        	18-8cc5c
Substitution matrix                    	aa:blosum62.out,nucl:nucleotide.out
Add backtrace                          	false
Alignment mode                         	3
Alignment mode                         	0
Allow wrapped scoring                  	false
E-value threshold                      	0.001
Seq. id. threshold                     	0
Min alignment length                   	0
Seq. id. mode                          	0
Alternative alignments                 	0
Coverage threshold                     	0
Coverage mode                          	0
Max sequence length                    	65535
Compositional bias                     	1
Compositional bias scale               	1
Max reject                             	2147483647
Max accept                             	2147483

In [8]:
!rm -r tmp

In [2]:
import pandas as pd

In [3]:
#df = pd.read_csv('vsgsSearch/res_vsg.m8',sep='\t')
#for i,n in enumerate(df['query']):
#    print(i,n,n.split('_')[1])
df = pd.read_csv('vsgsSearch/res_vsg.m8',sep='\t')
df['VSG_CLASS']=[n.split('_')[1].split('_')[0]  for n in df['query']]
df['VSG_CLASS'].unique()

array(['MC', 'array', 'BES', 'MES'], dtype=object)

In [4]:
df = pd.read_csv('vsgsSearch/res_vsg.m8',sep='\t')
df.sort_values('evalue')

,query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits
0,Tb427VSG-600_MC,Tb427VSG-600_unitig_Tb427v10,1.000,1200,0,0,1,1200,5115,6314,0.000000,2151
1227,Tb427VSG-324_array_Chr2_5A_Tb427v10:Chr9_3A_Tb...,Chr6_3A_Tb427v10,0.900,1200,120,0,1200,1,1282067,1283263,0.000000,1610
1228,Tb427VSG-324_array_Chr2_5A_Tb427v10:Chr9_3A_Tb...,Chr1_3A_Tb427v10,0.889,1200,133,0,1200,1,143859,145058,0.000000,1547
1229,Tb427VSG-324_array_Chr2_5A_Tb427v10:Chr9_3A_Tb...,Chr11_3A_Tb427v10,0.869,1200,157,0,1,1200,100042,101241,0.000000,1446
1230,Tb427VSG-324_array_Chr2_5A_Tb427v10:Chr9_3A_Tb...,Chr4_3A_Tb427v10,0.807,1200,231,0,1200,1,576627,577826,0.000000,1112
...,...,...,...,...,...,...,...,...,...,...,...,...
2328,Tb427VSG-449_array_Chr9_5B_Tb427v10,Chr1_3A_Tb427v10,0.850,40,6,0,631,592,457404,457443,0.000674,46
2304,Tb427VSG-476_MC,Chr1_3A_Tb427v10,0.885,35,4,0,1134,1100,494371,494405,0.000674,46
2303,Tb427VSG-476_MC,Chr7_5A_Tb427v10,0.885,35,4,0,1099,1133,554618,554652,0.000674,46
140,Tb427VSG-672_MC,Chr5_3B_Tb427v10,0.770,61,13,0,1196,1139,212005,212065,0.000674,46


In [5]:
df = pd.read_csv('vsgsSearch/res_vsg.m8',sep='\t')
df['VSG_CLASS']=[n.split('_')[1].split('_')[0]  for n in df['query']]
df = df[(df['VSG_CLASS']=='array') & (df['target'].str.contains('unitig'))]
print(len(df['query'].unique()))
df.sort_values('evalue').head()
open('exclude_array.txt','w').write('\n'.join( [n.split('_')[0] for n in list(df['query'].unique())] ))

26


337

In [7]:
df = pd.read_csv('vsgsSearch/res_vsg.m8',sep='\t')
df['VSG_CLASS']=[n.split('_')[1].split('_')[0]  for n in df['query']]
df = df[(df['VSG_CLASS']=='MC') & (df['target'].str.startswith('Chr'))]
print(len(df['query'].unique()))
df.head()
open('exclude_MC.txt','w').write('\n'.join( [n.split('_')[0] for n in list(df['query'].unique())] ))

59


774

In [14]:
df = pd.read_csv('vsgsSearch/res_vsg.m8',sep='\t')
df['VSG_CLASS']=[n.split('_')[1].split('_')[0]  for n in df['query']]
df['VSG_CLASS'].value_counts()

VSG_CLASS
array    1306
MC       1231
BES       121
MES        20
Name: count, dtype: int64

In [15]:
df[~df['query'].duplicated(keep=False)]['VSG_CLASS'].value_counts()

VSG_CLASS
array    38
MC        5
MES       1
BES       1
Name: count, dtype: int64

In [17]:
#!pwd

In [18]:
df = pd.read_csv('vsgsSearch/res_vsg.m8',sep='\t')
df['VSG_CLASS']=[n.split('_')[1].split('_')[0]  for n in df['query']]
df = df[(df['VSG_CLASS']=='array') ]
df.tail()

,query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,VSG_CLASS
2673,Tb427VSG-352_array_Chr9_3A_Tb427v10:Chr8_5A_Tb...,Chr1_5A_Tb427v10,0.913,1200,104,0,1,1200,112703,113902,0.0,1678,array
2674,Tb427VSG-352_array_Chr9_3A_Tb427v10:Chr8_5A_Tb...,Chr10_3A_Tb427v10,0.872,1200,153,0,1,1200,1203452,1204651,0.0,1466,array
2675,Tb427VSG-352_array_Chr9_3A_Tb427v10:Chr8_5A_Tb...,Chr1_3B_Tb427v10,0.872,1200,153,0,1200,1,554390,555589,0.0,1449,array
2676,Tb427VSG-367_array_Chr10_3B_Tb427v10,Chr10_3B_Tb427v10,1.000,1200,0,0,1,1200,124558,125757,0.0,2151,array
2677,Tb427VSG-431_array_Chr10_3B_Tb427v10,Chr10_3B_Tb427v10,1.000,1200,0,0,1200,1,800352,801551,0.0,2151,array


In [19]:
import json
with open('vsgs_web_server/data/exp_config.json', 'r') as f:
    exp_config = json.load(f)
all_treatments = []
for key in exp_config:
    all_treatments+=exp_config[key]['treatments']
len(all_treatments),all_treatments[0:3]

(216, ['SRR2154098', 'SRR2154094', 'SRR2154095'])

In [21]:
import os
def open_counts(temp_id):
    path = f'../myRna-seq/results/result_vsgs/{temp_id}'
    for fname in os.listdir(path):
        if fname.endswith('_all.txt'):
            tmp_df = pd.read_csv(f'{path}/{fname}',sep='\t',comment='#')
            cols =  list(tmp_df.columns)
            cols[-1]='counts'
            tmp_df.columns = cols
            tmp_df.set_index('Geneid',inplace=True)
            return tmp_df
all_treatments_counts = []

for temp_id in all_treatments:
    tmp_df = open_counts(temp_id)
    tmp_df = tmp_df[['counts']]
    tmp_df.columns=[temp_id]
    all_treatments_counts.append(tmp_df)
all_treatments_counts=pd.concat(all_treatments_counts,axis=1)
all_treatments_counts

,SRR2154098,SRR2154094,SRR2154095,SRR2154096,SRR2154091,SRR2154088,SRR2154092,SRR2154089,ERR3447799,ERR3447800,...,ERR3383644,ERR3383646,ERR3380229,ERR3380233,ERR3380232,ERR3380236,ERR3380231,ERR3380235,ERR3380230,ERR3380234
Geneid,,,,,,,,,,,,,,,,,,,,,
Tb927.1.05,0,12,1,1,0,0,0,2,1,1,...,5,7,8,8,0,5,1,1,0,0
Tb927.1.10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Tb927.1.30,14,20,6,2,3,10,5,7,11,7,...,2,7,8,8,0,1,1,0,0,1
Tb927.1.40,6,610,4,4,2,6,9,20,52,47,...,13,19,42,20,0,4,1,0,0,0
Tb927.1.70,822,1839,965,888,1003,1586,1699,1588,252,239,...,2467,1342,1242,1179,59,387,92,192,40,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lncRNA_curated_4102,1745,2928,1345,1169,1356,2004,2476,1380,251,225,...,1335,633,3320,2805,181,389,145,133,48,29
lncRNA_curated_4104,1367,2543,1329,1208,1414,2091,2303,1472,185,188,...,1561,618,4105,3369,214,665,183,170,49,38
lncRNA_curated_4105,1368,2543,1330,1208,1415,2092,2304,1473,185,188,...,1563,619,4106,3372,214,665,183,170,49,38


In [22]:
vsg_counts = all_treatments_counts[all_treatments_counts.index.str.contains('VSG')]
vsg_counts['has_value_gte_10'] = (vsg_counts >= 10).any(axis=1)
vsg_counts['has_value_gte_10'].value_counts()

/var/folders/c8/lvfcy73x5jl6yvb91241cvdh0000gp/T/ipykernel_94651/884445362.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  vsg_counts['has_value_gte_10'] = (vsg_counts >= 10).any(axis=1)
/var/folders/c8/lvfcy73x5jl6yvb91241cvdh0000gp/T/ipykernel_94651/884445362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vsg_counts['has_value_gte_10'] = (vsg_counts >= 10).any(axis=1)


has_value_gte_10
True     291
False      7
Name: count, dtype: int64

In [23]:
open('exclude_count.txt','w').write('\n'.join( [n.split('_')[1] for n in list(vsg_counts[~vsg_counts['has_value_gte_10']].index.values)] ))

90